<a href="https://colab.research.google.com/github/a00101/datascience101/blob/main/all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [139]:
df = pd.read_csv('input.txt',sep="\t")
X = df.iloc[:,1:]
Y = df.iloc[:,:1].values.flatten()
X.shape, Y.shape



((508, 14339), (508,))

In [140]:
X_shadow = X.apply(np.random.permutation) # Make X_shadow by randomly permuting each column of X
X_shadow.columns = ['shuffled_' + feat for feat in X.columns]
X_boruta = pd.concat([X, X_shadow], axis = 1)
X_boruta

,A1BG,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AAR2,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB10,ABCB4,ABCB6,ABCB7,ABCB8,...,shuffled_ZP1,shuffled_ZP3,shuffled_ZRANB1,shuffled_ZRANB2,shuffled_ZRANB3,shuffled_ZRSR2,shuffled_ZSCAN12,shuffled_ZSCAN16,shuffled_ZSCAN18,shuffled_ZSCAN2,shuffled_ZSCAN20,shuffled_ZSCAN21,shuffled_ZSCAN22,shuffled_ZSCAN23,shuffled_ZSCAN25,shuffled_ZSCAN26,shuffled_ZSCAN29,shuffled_ZSCAN30,shuffled_ZSCAN31,shuffled_ZSCAN32,shuffled_ZSCAN5A,shuffled_ZSCAN9,shuffled_ZSWIM1,shuffled_ZSWIM3,shuffled_ZSWIM4,shuffled_ZSWIM5,shuffled_ZSWIM6,shuffled_ZSWIM7,shuffled_ZSWIM8,shuffled_ZW10,shuffled_ZWILCH,shuffled_ZWINT,shuffled_ZXDA,shuffled_ZXDB,shuffled_ZXDC,shuffled_ZYG11A,shuffled_ZYG11B,shuffled_ZYX,shuffled_ZZEF1,shuffled_ZZZ3
0,-0.6408,-0.3098,1.3281,-0.0121,-0.0001,-0.8631,-0.8037,-0.4713,0.4541,-0.9616,0.3699,0.3349,-0.0700,0.7701,0.0023,-0.3226,-1.8136,0.4409,0.0266,0.6362,1.6727,0.1921,1.3617,1.4873,-1.9710,0.7132,0.8486,1.0824,0.7911,0.1268,2.3755,-0.5618,1.7515,2.2333,1.0838,-0.0392,-1.7623,-1.2639,-0.5647,-0.5423,...,-1.2518,1.4593,-0.2271,-1.8726,1.2897,-0.9848,0.0292,-0.1539,-1.3070,0.4004,0.8309,-2.2247,0.6878,-0.5718,0.3935,-1.1116,0.2056,0.2955,0.7086,0.0007,-0.3479,0.2794,1.0242,-0.1951,0.5991,-2.5289,0.2609,1.7714,-0.7420,0.8561,-0.4439,-2.1498,0.5024,-0.5267,-1.3592,0.3709,-0.2534,-1.2305,1.0420,1.4035
1,0.8489,-0.1820,0.8837,-0.5358,-0.2023,0.2091,-0.5103,0.2144,1.1760,-1.8115,0.1731,0.4357,-0.9291,-1.5382,-2.4188,-1.3764,0.2823,-2.0997,-0.5570,-1.2794,0.1512,-1.1089,-2.4153,0.5204,-0.2975,0.8727,-1.0772,-0.7522,1.1641,0.4426,0.0648,-0.9140,0.1058,-0.0970,-0.4115,-0.3881,0.2120,0.5478,-0.8341,-0.0845,...,1.0398,1.8872,0.5714,0.7427,0.3366,-0.3265,0.1460,0.7388,0.6689,0.2072,-0.4450,-0.4256,-0.2918,0.7082,-0.2245,-0.3154,-0.0849,-0.2237,-0.3995,-0.2188,-0.1666,1.2305,0.5708,0.7894,0.1530,-0.6283,0.1438,1.3739,1.3825,0.7879,-0.9627,0.5786,1.3534,-1.2560,1.7443,0.8416,-1.3534,0.5003,-1.0480,0.0890
2,0.2019,0.5909,-0.1186,-0.4136,-0.5202,0.2144,-0.7461,1.2570,0.2573,-1.1025,0.6609,-0.3344,-1.6926,-1.3337,-0.9939,0.2351,1.6618,-0.1939,-0.0946,0.2744,0.0890,0.3744,-0.0113,0.4174,-0.4109,0.4890,-0.6342,0.1133,0.1648,1.4228,0.0491,1.4474,0.0159,-0.0203,-0.9883,0.6209,-0.2176,0.7721,-1.1790,0.7181,...,-1.6542,1.4312,0.0025,0.7363,-0.0007,0.0298,-0.4804,0.1479,0.1496,1.0883,-0.7658,-0.3426,-0.1168,1.2785,0.2809,0.5498,0.0147,-0.2183,-1.0563,-0.3256,-0.8174,-0.0817,0.3313,-0.9576,-0.6832,0.4273,1.0499,-1.2333,0.0188,-0.7133,0.6209,1.3432,-0.5179,0.4060,0.8243,0.7281,0.5706,-0.6271,0.9269,1.3104
3,1.7515,-0.2773,0.8486,-0.7224,-0.0614,0.5340,-1.2040,0.1164,2.4483,-0.6460,1.7535,-0.2597,-0.6467,-2.2880,-2.4776,-1.2281,-0.0628,-0.0689,-1.0269,0.1369,2.0708,-0.5731,-0.7762,2.2164,-0.1877,0.5621,-0.8331,0.0923,0.9818,2.0234,1.0993,2.6222,0.9530,0.1955,0.5243,-0.0916,0.0378,-0.2079,-0.6517,0.6251,...,-1.3215,0.7947,0.6598,0.3625,-0.5186,-1.5094,0.3539,-0.0627,-0.8768,-0.6015,1.1905,0.7732,0.5486,-0.9103,0.0651,2.2308,0.3495,-0.4104,0.7761,0.1736,0.7048,3.0487,0.1896,-0.3020,-0.6002,-0.4828,0.3923,0.3061,-0.0617,0.4202,0.9215,0.1072,0.2329,-0.5735,-0.9057,-1.3990,-0.4389,-0.0012,-0.4245,2.2039
4,1.4047,-0.2160,-0.8791,0.0195,0.3124,1.1492,-0.1435,0.3805,0.9215,-0.8840,-0.1578,-0.1130,-1.5736,-1.0002,-0.8065,-0.3168,1.0035,-1.0457,0.0266,0.1620,-0.5467,-0.1257,-3.1990,0.0568,-0.0247,0.8129,0.4417,0.1719,-0.0047,1.1371,-0.9699,0.5504,-0.4600,-0.4589,-0.8415,-0.9026,-0.0311,0.5007,-0.3307,0.0454,...,-0.9941,-1.3789,-0.1198,-0.9857,0.0478,-0.8099,1.1929,0.0260,-0.6140,-1.3175,-0.8850,0.9772,0.3576,1.8925,-0.8353,-0.0221,-0.1534,0.3968,1.4782,-0.0644,-0.7854,0.5512,0.1286,-0.2208,0.0305,1.2854,-0.3869,0.2991,0.2639,-0.3105,-1.2232,-0.1696,-0.5484,0.5907,-0.1722,1.2323,0.8032,-0.9422,-0.8126,0.5072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
mdl = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
mdl.fit(X_boruta, Y)



In [142]:
feature_imp_X = mdl.feature_importances_
feature_imp_shuffled = mdl.feature_importances_[len(X.columns):]


In [149]:
feature_imp_shuffled = [round(i*100,2) for i in feature_imp_shuffled]
feature_imp_X = [round(i*100,2) for i in feature_imp_X]
print('Threshold for feature importance:',max(feature_imp_shuffled))

baga = []
for i in range(len(X_boruta.columns)):
    if mdl.feature_importances_[i] < max(feature_imp_shuffled):
      baga.append(mdl.feature_importances_[i])
print('From real data: ', max(baga))
      # print(X_boruta.columns[i]+': '+str(mdl.feature_importances_[i]))


Threshold for feature importance: 850000000000.0
From real data:  0.04839432535685623


In [132]:
hits = np.zeros((len(X.columns)))

trials = 30
## Perform 30 trials
for i in range(trials):
   ## Create X_shuffle
   np.random.seed(i)
   X_shuffle = X.apply(np.random.permutation)
   X_boruta = pd.concat([X, X_shuffle], axis = 1)
   ## Fit random forest
   mdl = RandomForestClassifier(n_estimators=5, random_state=0)
   mdl.fit(X_boruta, Y)
   ## Store feature importance
   feature_imp_X = mdl.feature_importances_[:len(X.columns)]
   feature_imp_shuffled = mdl.feature_importances_[len(X.columns):]
   ## Compute hits for this trial and add to counter
   hits += (feature_imp_X > feature_imp_shuffled.max())
print('Hits for every feature')
for i in range(len(X.columns)):
    if hits[i] > trials/2:
      print(X.columns[i],':',hits[i])
    else:
      pass

Hits for every feature


In [137]:
import scipy as sp
trials = 100
pmf = [sp.stats.binom.pmf(x, trials, .5) for x in range(1,trials + 1)]
import matplotlib.pyplot as plt

plt.xticks(np.arange(0, 101, 1)) 
plt.plot(range(1,6),pmf[:5],'-ro')
plt.plot(range(6,25),pmf[5:24],'-bo')
plt.plot(range(25,31),pmf[24:30],'-go')

plt.fill_between(range(1,6),pmf[:5] , color='red')
plt.fill_between(range(6,25),pmf[5:24] , color='blue')
plt.fill_between(range(25,101),pmf[24:30] , color='green')

plt.title('Plot for binomial distribution of features')
plt.xlabel('Instance count when features were important')
plt.ylabel('Binomial distribution values')
labels = ['Discard','Might be important','Keep']
plt.legend(labels,loc='upper left')

[7.888609052209954e-29,
 3.904861480843993e-27,
 1.2755880837423357e-25,
 3.0933011030751746e-24,
 5.939138117904783e-23,
 9.403635353348883e-22,
 1.2627738903067746e-20,
 1.4679746474816387e-19,
 1.5005963063145872e-18,
 1.3655426387462894e-17,
 1.11726215897433e-16,
 8.286361012392477e-16,
 5.609228993003864e-15,
 3.485735159938344e-14,
 1.9984881583646665e-13,
 1.0616968341312046e-12,
 5.2460314157070374e-12,
 2.419003375020342e-11,
 1.0439909302719236e-10,
 4.228163267601253e-10,
 1.6107288638483196e-09,
 5.783980920182195e-09,
 1.9615239642356016e-08,
 6.293222718589583e-08,
 1.9131397064512463e-07,
 5.518672230147915e-07,
 1.5125249815959707e-06,
 3.943368702018246e-06,
 9.79043263949372e-06,
 2.317069058013523e-05,
 5.232091421320581e-05,
 0.00011281697127222451,
 0.000232471334742764,
 0.00045810527728724286,
 0.0008638556657416551,
 0.0015597393964780048,
 0.002697927604718661,
 0.004472879976244036,
 0.007110732269926419,
 0.010843866711637111,
 0.015869073236543515,
 0.02229